# Phase 3 Weighted Bagging

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

import os
import re
import csv
import codecs
import gensim
import itertools
import numpy as np
import pandas as pd
import operator
import sys

from nltk import ngrams
from collections import Counter
from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.config import dataset_config
from iwillwin.data_utils.feature_engineering import FeatureCreator

from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize

import seaborn as sns
%matplotlib inline

import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

import os
import re
import csv
import codecs
import numpy as np
import pandas as pd
import operator
from os import listdir
from os.path import isfile, join

########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import pandas as pd
import operator
import sys

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.config import dataset_config
from keras.utils import to_categorical

C:\Users\zake7\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
C:\Users\zake7\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
NB_WORDS, MAX_SEQUENCE_LENGTH = 50000, 50
data_transformer = DataTransformer(max_num_words=NB_WORDS, max_sequence_length=MAX_SEQUENCE_LENGTH, char_level=False,
                                   normalization=True, features_processed=True)
trains_nns, tests_nns, labels = data_transformer.prepare_data(dual=False)
print("Number of unique words", len(data_transformer.tokenizer.index_docs))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\zake7\AppData\Local\Temp\jieba.cache
Loading model cost 0.846 seconds.
Prefix dict has been built succesfully.


[DataHelper] Apply normalization on value-type columns


C:\Users\zake7\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Doing preprocessing...
Transforming words to indices...
Shape of data tensor: (320552, 50) (320552, 50)
Shape of label tensor: (320552,)
Preprocessed.
Number of unique words 83265


In [3]:
trains_meta = trains_nns[2]
tests_meta = tests_nns[2]

In [4]:
train_df = pd.read_csv('../data/dataset/train.csv')
test_df = pd.read_csv('../data/dataset/test.csv')

In [5]:
rumor_words = ['辟谣', '谣言', '勿传', '假的']

def is_rumor(text):
    if type(text) != str:
        print(text, type(text))
        return 0
    for rumor_word in rumor_words:
        if rumor_word in text:
            return 1
    return 0

def has_split_symbol(text):
    if type(text) != str:
        return 0
    if '|' in text:
        return 1
    return 0

for df in [train_df, test_df]:
    df['has_|'] = df['title2_zh'].apply(has_split_symbol)
    df['has_rumor_words'] = df['title2_zh'].apply(is_rumor)

nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>


In [6]:
train_has_rumor = train_df.has_rumor_words.values
test_has_rumor = test_df.has_rumor_words.values

trick_trains_features = np.concatenate((trains_nns[2], train_has_rumor.reshape((-1, 1))), axis=1)
trick_tests_features = np.concatenate((tests_nns[2], test_has_rumor.reshape((-1, 1))), axis=1)

In [7]:
oof_file_names = sorted([f for f in listdir('../data/pseudo/oofs/') if isfile(join('../data/pseudo/oofs/', f)) and f != '.gitkeep'])
preds_file_names = [name.replace('-Train', '') for name in oof_file_names]

oofs = []
preds = []
for name in oof_file_names:
    oofs.append(pd.read_csv('../data/pseudo/oofs/' + name))
for name in preds_file_names:
    preds.append(pd.read_csv('../data/pseudo/output/' + name))    

In [8]:
for i, name in enumerate(oof_file_names):
    print(i, name)
    
trains = pd.DataFrame()
tests = pd.DataFrame()

for i in range(len(oof_file_names)):
    for label_type in ['agreed', 'disagreed', 'unrelated']:
        trains['oofs_{}_{}'.format(i, label_type)] = oofs[i][label_type].values
        tests['oofs_pred{}_{}'.format(i, label_type)] = preds[i][label_type].values

0 3Embedding-DecomposalbeAttention-NoMeta-ClassWeighted-NoEM-Train-L0.857551-NB5000.csv
1 P3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM-Train-L0.297362-NB5000.csv
2 P3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM-Train-L0.292235-NB5000.csv
3 P3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM-Train-L0.283131-NB5000.csv
4 PS3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM-Train-L0.853793-NB5000.csv
5 PS3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM-Train-L0.853764-NB5000.csv
6 PS3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM-Train-L0.863459-NB5000.csv
7 PSWordSGNS-DAttn-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.861867-NB100000.csv
8 PSWordSGNS-DenseCNN5Layers-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.851927-NB100000.csv
9 PSWordSGNS-DenseCNN5Layers-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.855071-NB100000.csv
10 PSWordSGNS-ESIM-NoMeta-3P-NoEM-NoClassWeighted-3Layers-Train-L0.860951-NB100000.csv
11 PSWord

In [9]:
unrelated = pd.DataFrame()
agreeds = pd.DataFrame()
disagreeds = pd.DataFrame()

#check_oofs = True
check_oofs = False


if check_oofs:
    for i, oof in enumerate(oofs):
        agreeds['oofs_agreed_{}'.format(i)] = oofs[i]['agreed'].values
        unrelated['oofs_unrelated_{}'.format(i)] = oofs[i]['unrelated'].values
        disagreeds['oofs_disagreeds_{}'.format(i)] = oofs[i]['disagreed'].values
else:
    for i, oof in enumerate(oofs):
        agreeds['oofs_agreed_{}'.format(i)] = preds[i]['agreed'].values
        unrelated['oofs_unrelated_{}'.format(i)] = preds[i]['unrelated'].values
        disagreeds['oofs_disagreeds_{}'.format(i)] = preds[i]['disagreed'].values  

In [10]:
agreeds.corr()

,oofs_agreed_0,oofs_agreed_1,oofs_agreed_2,oofs_agreed_3,oofs_agreed_4,oofs_agreed_5,oofs_agreed_6,oofs_agreed_7,oofs_agreed_8,oofs_agreed_9,oofs_agreed_10,oofs_agreed_11,oofs_agreed_12,oofs_agreed_13,oofs_agreed_14,oofs_agreed_15,oofs_agreed_16,oofs_agreed_17,oofs_agreed_18
oofs_agreed_0,1.000000,0.985454,0.988323,0.986448,0.984380,0.987315,0.985907,0.973378,0.975002,0.977427,0.975342,0.976909,0.976050,0.976170,0.976952,0.976174,0.975576,0.975719,0.976823
oofs_agreed_1,0.985454,1.000000,0.988241,0.986295,0.995884,0.988644,0.985872,0.976543,0.978513,0.977921,0.978879,0.974114,0.978497,0.978545,0.977649,0.979854,0.979520,0.978561,0.979853
oofs_agreed_2,0.988323,0.988241,1.000000,0.987435,0.988193,0.996147,0.987102,0.970575,0.976039,0.976866,0.977199,0.973728,0.978022,0.977212,0.976467,0.977935,0.976717,0.977700,0.977855
oofs_agreed_3,0.986448,0.986295,0.987435,1.000000,0.984308,0.986038,0.996268,0.976276,0.979169,0.978624,0.983136,0.979642,0.983276,0.984887,0.978003,0.984594,0.979907,0.983532,0.985561
oofs_agreed_4,0.984380,0.995884,0.988193,0.984308,1.000000,0.988589,0.984010,0.974411,0.977064,0.976446,0.976906,0.971661,0.976735,0.976505,0.976157,0.977627,0.977743,0.976696,0.977415
oofs_agreed_5,0.987315,0.988644,0.996147,0.986038,0.988589,1.000000,0.986175,0.971035,0.974898,0.975453,0.976168,0.971536,0.976450,0.975633,0.974975,0.976913,0.975815,0.976077,0.976816
oofs_agreed_6,0.985907,0.985872,0.987102,0.996268,0.984010,0.986175,1.000000,0.975417,0.977783,0.977023,0.981811,0.977934,0.981879,0.983655,0.976346,0.983055,0.978560,0.982419,0.984168
oofs_agreed_7,0.973378,0.976543,0.970575,0.976276,0.974411,0.971035,0.975417,1.000000,0.988611,0.982953,0.986930,0.983626,0.979390,0.982267,0.982601,0.986865,0.989011,0.979822,0.983091
oofs_agreed_8,0.975002,0.978513,0.976039,0.979169,0.977064,0.974898,0.977783,0.988611,1.000000,0.987310,0.990456,0.983518,0.984780,0.985321,0.987066,0.990289,0.997975,0.984720,0.985544
oofs_agreed_9,0.977427,0.977921,0.976866,0.978624,0.976446,0.975453,0.977023,0.982953,0.987310,1.000000,0.984884,0.989911,0.989481,0.987805,0.998649,0.985065,0.987453,0.989079,0.988042


In [11]:
disagreeds.corr()

,oofs_disagreeds_0,oofs_disagreeds_1,oofs_disagreeds_2,oofs_disagreeds_3,oofs_disagreeds_4,oofs_disagreeds_5,oofs_disagreeds_6,oofs_disagreeds_7,oofs_disagreeds_8,oofs_disagreeds_9,oofs_disagreeds_10,oofs_disagreeds_11,oofs_disagreeds_12,oofs_disagreeds_13,oofs_disagreeds_14,oofs_disagreeds_15,oofs_disagreeds_16,oofs_disagreeds_17,oofs_disagreeds_18
oofs_disagreeds_0,1.000000,0.956195,0.965381,0.967006,0.970607,0.973446,0.969427,0.947563,0.953924,0.953973,0.953546,0.955650,0.952962,0.952829,0.948303,0.949407,0.948455,0.949762,0.947914
oofs_disagreeds_1,0.956195,1.000000,0.972663,0.969680,0.980875,0.961191,0.958854,0.940807,0.950593,0.941773,0.946255,0.942006,0.941758,0.944460,0.953687,0.960452,0.958069,0.955050,0.956554
oofs_disagreeds_2,0.965381,0.972663,1.000000,0.973263,0.974988,0.984404,0.967257,0.938211,0.951614,0.948799,0.949472,0.945175,0.947688,0.950114,0.955320,0.955833,0.955834,0.956771,0.954623
oofs_disagreeds_3,0.967006,0.969680,0.973263,1.000000,0.973872,0.970303,0.986330,0.955033,0.962220,0.955903,0.963340,0.957298,0.961192,0.963206,0.959059,0.969456,0.963931,0.967005,0.968558
oofs_disagreeds_4,0.970607,0.980875,0.974988,0.973872,1.000000,0.976251,0.972630,0.948467,0.960164,0.957989,0.960040,0.955228,0.957282,0.957418,0.956558,0.959968,0.958002,0.959996,0.956790
oofs_disagreeds_5,0.973446,0.961191,0.984404,0.970303,0.976251,1.000000,0.971418,0.938822,0.953536,0.955377,0.955229,0.950740,0.954774,0.956343,0.949685,0.951494,0.949397,0.953315,0.948816
oofs_disagreeds_6,0.969427,0.958854,0.967257,0.986330,0.972630,0.971418,1.000000,0.954179,0.960763,0.957241,0.964684,0.959270,0.963397,0.965154,0.951198,0.961519,0.955037,0.960886,0.960835
oofs_disagreeds_7,0.947563,0.940807,0.938211,0.955033,0.948467,0.938822,0.954179,1.000000,0.975049,0.961024,0.967412,0.970211,0.958116,0.962381,0.960166,0.969288,0.969771,0.959363,0.964948
oofs_disagreeds_8,0.953924,0.950593,0.951614,0.962220,0.960164,0.953536,0.960763,0.975049,1.000000,0.976337,0.977972,0.971753,0.971154,0.973719,0.972987,0.978265,0.990127,0.972071,0.971182
oofs_disagreeds_9,0.953973,0.941773,0.948799,0.955903,0.957989,0.955377,0.957241,0.961024,0.976337,1.000000,0.972073,0.978640,0.978412,0.978475,0.987397,0.966143,0.969127,0.976464,0.970646


In [12]:
unrelated.corr()

,oofs_unrelated_0,oofs_unrelated_1,oofs_unrelated_2,oofs_unrelated_3,oofs_unrelated_4,oofs_unrelated_5,oofs_unrelated_6,oofs_unrelated_7,oofs_unrelated_8,oofs_unrelated_9,oofs_unrelated_10,oofs_unrelated_11,oofs_unrelated_12,oofs_unrelated_13,oofs_unrelated_14,oofs_unrelated_15,oofs_unrelated_16,oofs_unrelated_17,oofs_unrelated_18
oofs_unrelated_0,1.000000,0.979483,0.982918,0.982116,0.980407,0.983841,0.982145,0.967810,0.969136,0.971615,0.969398,0.971206,0.969555,0.970029,0.970348,0.969500,0.968974,0.969428,0.970400
oofs_unrelated_1,0.979483,1.000000,0.985805,0.983235,0.993880,0.983194,0.981397,0.970652,0.972543,0.969819,0.972554,0.966131,0.969558,0.970829,0.973004,0.975851,0.975135,0.973627,0.975486
oofs_unrelated_2,0.982918,0.985805,1.000000,0.984631,0.985170,0.992641,0.983242,0.964436,0.969656,0.968727,0.970621,0.965539,0.969039,0.969442,0.971613,0.973276,0.971808,0.972618,0.973165
oofs_unrelated_3,0.982116,0.983235,0.984631,1.000000,0.981340,0.981809,0.994782,0.971740,0.974504,0.972338,0.978712,0.973816,0.977123,0.979705,0.973593,0.981349,0.975971,0.979870,0.982335
oofs_unrelated_4,0.980407,0.993880,0.985170,0.981340,1.000000,0.985285,0.980849,0.969350,0.972134,0.970553,0.971981,0.965403,0.970200,0.970659,0.971251,0.973084,0.973036,0.972064,0.972748
oofs_unrelated_5,0.983841,0.983194,0.992641,0.981809,0.985285,1.000000,0.982544,0.964530,0.968744,0.969367,0.970257,0.964712,0.970017,0.969717,0.968046,0.970220,0.969092,0.969949,0.970316
oofs_unrelated_6,0.982145,0.981397,0.983242,0.994782,0.980849,0.982544,1.000000,0.970761,0.973068,0.971318,0.977646,0.972614,0.976537,0.979104,0.970859,0.978754,0.973566,0.978102,0.979990
oofs_unrelated_7,0.967810,0.970652,0.964436,0.971740,0.969350,0.964530,0.970761,1.000000,0.986337,0.978792,0.983965,0.980390,0.974268,0.978164,0.978848,0.983940,0.986260,0.975830,0.979610
oofs_unrelated_8,0.969136,0.972543,0.969656,0.974504,0.972134,0.968744,0.973068,0.986337,1.000000,0.984056,0.987820,0.979701,0.980506,0.981747,0.983653,0.987454,0.996736,0.981314,0.982084
oofs_unrelated_9,0.971615,0.969819,0.968727,0.972338,0.970553,0.969367,0.971318,0.978792,0.984056,1.000000,0.981037,0.987146,0.986571,0.984970,0.996133,0.979866,0.982942,0.985613,0.983777


# Prepare Different Inputs

In [13]:
# Only use oofs
ensemble_trains = trains.values
ensemble_tests = tests.values

# Ensemble With NN

In [14]:
########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import pandas as pd
import operator
import sys

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.model.sim_zoos import *
import tensorflow as tf
from keras.layers import Dense, Input, MaxPooling1D, CuDNNLSTM, Embedding, Add, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from iwillwin.config import dataset_config, model_config
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Lambda, Dense, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.layers.wrappers import Bidirectional
from keras.legacy.layers import Highway
from keras.layers import TimeDistributed
from keras.layers.normalization import BatchNormalization
import keras.backend as K

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import log_loss

In [15]:
from keras import regularizers

def get_dense_add_net(feature_nums):
    features_inputs = Input(shape=(feature_nums,), name='mata-features', dtype="float32")
    features = features_inputs
    
    depth = 5
    for i in range(depth):
        new_features = Dense(24, activation='relu')(features)
        new_features = Dropout(0.1)(new_features)
        features = Concatenate()([features, new_features])

    h = Highway(activation='relu')(features)
    out_ = Dense(3, activation='softmax')(h)
    
    model = Model(inputs=[features_inputs], outputs=out_)
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [16]:
def get_logit_net(feature_nums):
    features_inputs = Input(shape=(feature_nums,), name='mata-features', dtype="float32")    
    out_ = Dense(3, activation='softmax')(features_inputs)
    model = Model(inputs=[features_inputs], outputs=out_)
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [17]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import importlib

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint

from iwillwin.config import model_config

class ModelTrainer(object):

    def __init__(self, model_stamp, epoch_num, learning_rate=1e-3,
                 shuffle_inputs=False, verbose_round=40, early_stopping_round=8):
        self.models = []
        self.model_stamp = model_stamp
        self.val_loss = -1
        self.auc = -1
        self.epoch_num = epoch_num
        self.learning_rate = learning_rate
        self.eps = 1e-10
        self.verbose_round = verbose_round
        self.early_stopping_round = early_stopping_round
        self.shuffle_inputs = shuffle_inputs
        self.class_weight = [0.93, 1.21]

    def train_folds(self, features, y, fold_count, batch_size, get_model_func, augments=None, skip_fold=0, patience=10, scale_sample_weight=False,
                    class_weight=None, self_aware=False, swap_input=False, early_stop_on=None):
        weight_val=scale_sample_weight
        fold_size = len(features) // fold_count
        models = []
        fold_predictions = []
        score = 0

        for fold_id in range(0, fold_count):
            fold_start = fold_size * fold_id
            fold_end = fold_start + fold_size

            if fold_id == fold_count - 1:
                fold_end = len(features)

            train_features = np.concatenate([features[:fold_start], features[fold_end:]])
            train_y = np.concatenate([y[:fold_start], y[fold_end:]])
            
            val_features = features[fold_start:fold_end]
            val_y = y[fold_start:fold_end]
            fold_pos = (np.sum(train_y) / len(train_features))

            train_data = {
                "mata-features": train_features,
            }

            val_data = {
                "mata-features": val_features,
            }

            model, bst_val_score, fold_prediction = self._train_model_by_logloss(
                get_model_func(), batch_size, train_data, train_y, val_data, val_y, fold_id,
                patience, class_weight, early_stop_on, weight_val=None)
    
            score += bst_val_score
            models.append(model)
            fold_predictions.append(fold_prediction)

        self.models = models
        self.val_loss = score / fold_count
        return models, self.val_loss, fold_predictions

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience):
        # return a list which holds [models, val_loss, auc, prediction]
        raise NotImplementedError

class KerasModelTrainer(ModelTrainer):

    def __init__(self, *args, **kwargs):
        super(KerasModelTrainer, self).__init__(*args, **kwargs)
        pass

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience, class_weight, early_stop_on, weight_val):
        early_stopping = EarlyStopping(monitor=early_stop_on, patience=10)
        bst_model_path = self.model_stamp + str(fold_id) + '.h5'
        val_data = (val_x, val_y, weight_val) if weight_val is not None else (val_x, val_y)
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
        hist = model.fit(train_x, train_y,
                         validation_data=val_data,
                         epochs=self.epoch_num, batch_size=batch_size, shuffle=True,
                         verbose=2, class_weight=class_weight,
                         callbacks=[early_stopping, model_checkpoint],)
        bst_val_score = max(hist.history['val_weighted_accuracy'])
        model.load_weights(bst_model_path)
        predictions = model.predict(val_x)

        return model, bst_val_score, predictions

In [18]:
def _agent_get_model():
    return get_dense_add_net(ensemble_trains.shape[1])

In [19]:
def np_weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    weight_mask = np.max(weight_mask, axis=-1)
    norms = np.sum(weight_mask)
    
    y_true = np.argmax(y_true, axis=-1)
    y_pred = np.argmax(y_pred, axis=-1)
    
    res = ((y_true == y_pred) * weight_mask).sum() / norms
    return res

In [20]:
def weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    label_weights = K.max(K.cast(weight_mask, 'float32'), axis=-1)
    
    true_label = K.argmax(y_true, axis=-1)
    pred_label = K.argmax(y_pred, axis=-1)
    
    res = K.cast(K.equal(true_label, pred_label), tf.float32) * label_weights / K.sum(label_weights)
    res = K.sum(res)
    return res

# Scale, Early Stopping on WACC

In [21]:
trainer = KerasModelTrainer(model_stamp="Ensemble-DenseNet", epoch_num=500)
models, score, folds_preds = trainer.train_folds(features=ensemble_trains, y=to_categorical(labels), augments=None, fold_count=10,
    batch_size=1024, early_stop_on='val_weighted_accuracy',
    scale_sample_weight=None, class_weight={0: 1/16, 1:1/15, 2:1/5},
    get_model_func=_agent_get_model, 
    patience=20)

C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mata-features (InputLayer)      (None, 57)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 24)           1392        mata-features[0][0]              
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 24)           0           dense_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 81)           0           mata-features[0][0]              
                                                                 dropout_1[0][0]                  
__________

concatenate_14 (Concatenate)    (None, 153)          0           concatenate_13[0][0]             
                                                                 dropout_14[0][0]                 
__________________________________________________________________________________________________
dense_17 (Dense)                (None, 24)           3696        concatenate_14[0][0]             
__________________________________________________________________________________________________
dropout_15 (Dropout)            (None, 24)           0           dense_17[0][0]                   
__________________________________________________________________________________________________
concatenate_15 (Concatenate)    (None, 177)          0           concatenate_14[0][0]             
                                                                 dropout_15[0][0]                 
__________________________________________________________________________________________________
highway_3 

Epoch 3/500
 - 2s - loss: 0.0187 - acc: 0.8875 - weighted_accuracy: 0.8793 - val_loss: 0.2747 - val_acc: 0.8788 - val_weighted_accuracy: 0.8725
Epoch 4/500
 - 2s - loss: 0.0187 - acc: 0.8874 - weighted_accuracy: 0.8794 - val_loss: 0.2764 - val_acc: 0.8779 - val_weighted_accuracy: 0.8720
Epoch 5/500
 - 2s - loss: 0.0187 - acc: 0.8871 - weighted_accuracy: 0.8793 - val_loss: 0.2797 - val_acc: 0.8767 - val_weighted_accuracy: 0.8725
Epoch 6/500
 - 2s - loss: 0.0186 - acc: 0.8869 - weighted_accuracy: 0.8788 - val_loss: 0.2760 - val_acc: 0.8773 - val_weighted_accuracy: 0.8726
Epoch 7/500
 - 2s - loss: 0.0186 - acc: 0.8875 - weighted_accuracy: 0.8795 - val_loss: 0.2696 - val_acc: 0.8801 - val_weighted_accuracy: 0.8727
Epoch 8/500
 - 2s - loss: 0.0186 - acc: 0.8875 - weighted_accuracy: 0.8795 - val_loss: 0.2765 - val_acc: 0.8779 - val_weighted_accuracy: 0.8724
Epoch 9/500
 - 2s - loss: 0.0186 - acc: 0.8874 - weighted_accuracy: 0.8795 - val_loss: 0.2665 - val_acc: 0.8814 - val_weighted_accuracy:

Epoch 16/500
 - 2s - loss: 0.0185 - acc: 0.8882 - weighted_accuracy: 0.8801 - val_loss: 0.2663 - val_acc: 0.8834 - val_weighted_accuracy: 0.8757
Epoch 17/500
 - 2s - loss: 0.0185 - acc: 0.8881 - weighted_accuracy: 0.8799 - val_loss: 0.2633 - val_acc: 0.8846 - val_weighted_accuracy: 0.8757
Epoch 18/500
 - 2s - loss: 0.0185 - acc: 0.8883 - weighted_accuracy: 0.8801 - val_loss: 0.2627 - val_acc: 0.8833 - val_weighted_accuracy: 0.8751
Epoch 19/500
 - 2s - loss: 0.0185 - acc: 0.8883 - weighted_accuracy: 0.8801 - val_loss: 0.2661 - val_acc: 0.8816 - val_weighted_accuracy: 0.8758
Epoch 20/500
 - 2s - loss: 0.0185 - acc: 0.8884 - weighted_accuracy: 0.8802 - val_loss: 0.2657 - val_acc: 0.8819 - val_weighted_accuracy: 0.8756
Epoch 21/500
 - 2s - loss: 0.0185 - acc: 0.8884 - weighted_accuracy: 0.8803 - val_loss: 0.2644 - val_acc: 0.8838 - val_weighted_accuracy: 0.8758
Epoch 22/500
 - 2s - loss: 0.0184 - acc: 0.8888 - weighted_accuracy: 0.8806 - val_loss: 0.2665 - val_acc: 0.8815 - val_weighted_ac

concatenate_43 (Concatenate)    (None, 129)          0           concatenate_42[0][0]             
                                                                 dropout_43[0][0]                 
__________________________________________________________________________________________________
dense_52 (Dense)                (None, 24)           3120        concatenate_43[0][0]             
__________________________________________________________________________________________________
dropout_44 (Dropout)            (None, 24)           0           dense_52[0][0]                   
__________________________________________________________________________________________________
concatenate_44 (Concatenate)    (None, 153)          0           concatenate_43[0][0]             
                                                                 dropout_44[0][0]                 
__________________________________________________________________________________________________
dense_53 (

In [22]:
print("score", score)

score 0.8793773095658487


In [23]:
print("score", score)
oofs_dir = "../data/p_ensemble/oofs/"
output_dir = "../data/p_ensemble/preds/"
onehot_pred_dir = "../data/p_ensemble/nn_one_hot/"

model_submit_prefix = "AddNN-Ensemble-Weighted-StopOnAcc"

oofs_path = oofs_dir + model_submit_prefix
output_path = output_dir + model_submit_prefix
one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

print("Predicting training results...")
train_predicts = np.concatenate(folds_preds, axis=0)
score = np_weighted_accuracy(to_categorical(labels), train_predicts)

oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
oofs.to_csv(submit_path, index=False)

print("Predicting testing results...")
test_predicts_list = []
for fold_id, model in enumerate(models):
    test_predicts = model.predict({"mata-features": ensemble_tests}, batch_size=128, verbose=1)
    test_predicts_list.append(test_predicts)

test_predicts = np.zeros(test_predicts_list[0].shape)
for fold_predict in test_predicts_list:
    test_predicts += fold_predict
test_predicts /= len(test_predicts_list)

test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
test_predicts.to_csv(submit_path, index=False) # 0.3343

print("Predicting labeled testing results...")
ids = pd.read_csv("../data/dataset/test.csv")
pred_labels = test_predicts.idxmax(axis=1)
sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
sub.to_csv(submit_path, index=False)

score 0.8793773095658487
Predicting training results...
Predicting testing results...
80126/80126 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 11us/step
Predicting labeled testing results...


In [24]:
score = np_weighted_accuracy(to_categorical(labels), train_predicts)

In [25]:
score

0.8774185929693885

# Not Scale, Early Stopping on Loss

In [27]:
trainer = KerasModelTrainer(model_stamp="Ensemble-DenseNet", epoch_num=500)
models, score, folds_preds = trainer.train_folds(features=ensemble_trains, y=to_categorical(labels), augments=None, fold_count=10,
    batch_size=1024, early_stop_on='val_loss',
    scale_sample_weight=None, class_weight=None,
    get_model_func=_agent_get_model, 
    patience=20)

C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mata-features (InputLayer)      (None, 57)           0                                            
__________________________________________________________________________________________________
dense_61 (Dense)                (None, 24)           1392        mata-features[0][0]              
__________________________________________________________________________________________________
dropout_51 (Dropout)            (None, 24)           0           dense_61[0][0]                   
__________________________________________________________________________________________________
concatenate_51 (Concatenate)    (None, 81)           0           mata-features[0][0]              
                                                                 dropout_51[0][0]                 
__________

__________________________________________________________________________________________________
dense_75 (Dense)                (None, 24)           2544        concatenate_62[0][0]             
__________________________________________________________________________________________________
dropout_63 (Dropout)            (None, 24)           0           dense_75[0][0]                   
__________________________________________________________________________________________________
concatenate_63 (Concatenate)    (None, 129)          0           concatenate_62[0][0]             
                                                                 dropout_63[0][0]                 
__________________________________________________________________________________________________
dense_76 (Dense)                (None, 24)           3120        concatenate_63[0][0]             
__________________________________________________________________________________________________
dropout_64

__________________________________________________________________________________________________
dropout_74 (Dropout)            (None, 24)           0           dense_88[0][0]                   
__________________________________________________________________________________________________
concatenate_74 (Concatenate)    (None, 153)          0           concatenate_73[0][0]             
                                                                 dropout_74[0][0]                 
__________________________________________________________________________________________________
dense_89 (Dense)                (None, 24)           3696        concatenate_74[0][0]             
__________________________________________________________________________________________________
dropout_75 (Dropout)            (None, 24)           0           dense_89[0][0]                   
__________________________________________________________________________________________________
concatenat

dense_99 (Dense)                (None, 24)           2544        concatenate_82[0][0]             
__________________________________________________________________________________________________
dropout_83 (Dropout)            (None, 24)           0           dense_99[0][0]                   
__________________________________________________________________________________________________
concatenate_83 (Concatenate)    (None, 129)          0           concatenate_82[0][0]             
                                                                 dropout_83[0][0]                 
__________________________________________________________________________________________________
dense_100 (Dense)               (None, 24)           3120        concatenate_83[0][0]             
__________________________________________________________________________________________________
dropout_84 (Dropout)            (None, 24)           0           dense_100[0][0]                  
__________

concatenate_92 (Concatenate)    (None, 105)          0           concatenate_91[0][0]             
                                                                 dropout_92[0][0]                 
__________________________________________________________________________________________________
dense_111 (Dense)               (None, 24)           2544        concatenate_92[0][0]             
__________________________________________________________________________________________________
dropout_93 (Dropout)            (None, 24)           0           dense_111[0][0]                  
__________________________________________________________________________________________________
concatenate_93 (Concatenate)    (None, 129)          0           concatenate_92[0][0]             
                                                                 dropout_93[0][0]                 
__________________________________________________________________________________________________
dense_112 

In [28]:
print("score", score)
oofs_dir = "../data/p_ensemble/oofs/"
output_dir = "../data/p_ensemble/preds/"
onehot_pred_dir = "../data/p_ensemble/nn_one_hot/"

model_submit_prefix = "AddNN-Ensemble-Weighted-StopOnLoss"

oofs_path = oofs_dir + model_submit_prefix
output_path = output_dir + model_submit_prefix
one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

print("Predicting training results...")
train_predicts = np.concatenate(folds_preds, axis=0)
score = np_weighted_accuracy(to_categorical(labels), train_predicts)

oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
oofs.to_csv(submit_path, index=False)

print("Predicting testing results...")
test_predicts_list = []
for fold_id, model in enumerate(models):
    test_predicts = model.predict({"mata-features": ensemble_tests}, batch_size=128, verbose=1)
    test_predicts_list.append(test_predicts)

test_predicts = np.zeros(test_predicts_list[0].shape)
for fold_predict in test_predicts_list:
    test_predicts += fold_predict
test_predicts /= len(test_predicts_list)

test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
test_predicts.to_csv(submit_path, index=False) # 0.3343

print("Predicting labeled testing results...")
ids = pd.read_csv("../data/dataset/test.csv")
pred_labels = test_predicts.idxmax(axis=1)
sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
sub.to_csv(submit_path, index=False)

score 0.873541072919297
Predicting training results...
Predicting testing results...
80126/80126 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 14us/step
Predicting labeled testing results...


In [29]:
score = np_weighted_accuracy(to_categorical(labels), train_predicts)

In [30]:
score

0.8670897024003137